In [ ]:
from functools import cached_property
from typing import Any, Dict, Tuple, TYPE_CHECKING

import cirq
import numpy as np
import quimb.tensor as qtn
from attrs import frozen
from numpy.typing import NDArray
from typing import * # TODO

from cirq_qubitization import cirq_infra
from cirq_qubitization.bloq_algos.and_bloq import And
from cirq_qubitization.bloq_algos.basic_gates import XGate, CNOT
from cirq_qubitization.bloq_algos.basic_gates.swap import CSwap
from cirq_qubitization.quantum_graph.bloq import Bloq
from cirq_qubitization.quantum_graph.composite_bloq import CompositeBloqBuilder, SoquetT
from cirq_qubitization.quantum_graph.fancy_registers import FancyRegister, FancyRegisters
from cirq_qubitization.quantum_graph.quantum_graph import Soquet
from cirq_qubitization.t_complexity_protocol import t_complexity, TComplexity

from cirq_qubitization.jupyter_tools import show_bloq

In [ ]:
from cirq_qubitization.bloq_algos.unary_iteration import *

from cirq_qubitization.bloq_algos.set_constant import SetConstant

In [ ]:
(10).bit_length()

In [ ]:
(8).bit_length()

In [ ]:
n_items = 10
system_bitsize = 32
bloqs = tuple(SetConstant(k=k, bitsize=system_bitsize) for k in range(n_items))
ui = IndexedBloq(
    selection_bitsize=(n_items-1).bit_length(),
    target_bitsize=system_bitsize,
    bloqs=bloqs,
    target_reg_name='x',
)
show_bloq(ui)

In [ ]:
cbloq = ui.decompose_bloq()
show_bloq(cbloq)

In [ ]:
from cirq_qubitization.quantum_graph.musical_score import get_musical_score_data, draw_musical_score, LineManager, dump_musical_score
from cirq_qubitization.quantum_graph.quantum_graph import LeftDangle, DanglingT
from cirq_qubitization.quantum_graph.util_bloqs import Split, Join
from cirq_qubitization.quantum_graph.fancy_registers import Side
import heapq

# class MyLineManager(LineManager):
#     def new_y(self, binst, reg, idx=None):
#         if binst is LeftDangle and reg.name == 'target':
#             self.available.remove(10)
#             return 10
        
#         if isinstance(binst, DanglingT):
#             return super().new_y(binst, reg, idx)
        
#         if isinstance(binst.bloq, Split):
#             i, = idx
#             m = 2*i + 2
#             self.available.remove(m)
#             return m
        
#         if isinstance(binst.bloq, Join):
#             heapq.heappush(self.available, self.split_y)
        
#         return super().new_y(binst, reg, idx)
    
#     def free(self, binst, reg, arr):
#         if isinstance(binst, DanglingT):
#             return super().free(binst, reg, arr)
        
#         if isinstance(binst.bloq, Split):
#             self.finish_hline(arr.y, arr.seq_x)
#             self.split_y = arr.y
#             # Don't make it available
#             return
            
#         return super().free(binst, reg, arr)
        

class MyLineManager(LineManager):
    def maybe_reserve(self, binst, reg, idx):
        if binst is LeftDangle and reg.name == 'selection':
            self.reserve_n(reg.bitsize*2, lambda b,r: b.bloq_is(Split))
            
        if binst.bloq_is(Split):
            if reg.side is Side.LEFT:
                self.reserve_n(1, lambda b, r: b.bloq_is(Join))
            if reg.side is Side.RIGHT:
                self.reserve_n(1, lambda b, r: True)
        

msd = get_musical_score_data(cbloq, MyLineManager())
fig, ax = draw_musical_score(msd)
fig.set_figwidth(0.4*40)
fig.tight_layout()

In [ ]:
dump_musical_score(msd, 'ui_decomp')